In [64]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import pad_sequences

In [65]:
import pandas as pd
import numpy as np
import tensorflow as tf

# 데이터 불러오기

- Human Labeling된 데이터인 SKT , 현대백화점, 현대모비스, S-Oil, 기업은행, 셀트리온 데이터 셋을 이용하여 각 정확도를 올린다.

- 순서
    1. 데이터 불러오기
    2. 데이터 분리
    3. Title 의 토큰화 및 불용어 제거
    4. Label 의 원핫인코딩
    5. 길이가 다른 title의 정형화
    6. LSTM을 통한 딥러닝
    7. 정확도 확인

In [66]:
df_skt = pd.read_csv('./trainingset/SKT_traindata_482.csv')
df_train_skt = df_skt.iloc[:200]
df_test_skt = df_skt.iloc[200:]

In [67]:
df_test_skt

,id,Title,label
200,0,"정부, 구글 지도 해외 반출 불허 구글이 보완책 수용 안해 (상보)",0
201,1,"미래부, 2차 유료방송 발전방안 공개토론회 개최",0
202,2,"SKT, 전용폰 구입 고객에 T1 롤드컵 우승 기념품 증정",1
203,3,'갤S7'사면 페이백 50만원 준다더니제2의 거성 모바일 사태?,0
204,4,케이블TV 가입자도 내년 1월부터 SKT 요금할인받는다,1
...,...,...,...
476,78,"스웨덴 에릭슨""4차 산업혁명 협력할 韓 스타트업 찾습니다 ,0\r\n4187,SKT...",1
477,79,"CJ헬로비전 작년 영업이익 59.4% 감소한 429억원…""M&A 등 불확실성 해소,...",1
478,80,SKT 2019년까지 11조원 투자,1
479,81,통신사 커버리지맵 정확도 SKT 1위 '오차 0%',1


In [68]:
df_mobis = pd.read_csv('./trainingset/현대모비스_traindata_200.csv', encoding = 'CP949')
# df_mobis

In [69]:
df_hb = pd.read_csv('./trainingset/현대백화점_traindata_200.csv')
# df_hb

In [70]:
df_S_Oil = pd.read_csv('./trainingset/S-Oil_traindata_200.csv')
df_S_Oil = df_S_Oil.head(200)
# df_S_Oil.head(200)

In [71]:
df_IBK = pd.read_csv('./trainingset/기업은행_traindata_200.csv')
# df_IBK

In [72]:
df_celt = pd.read_csv('./trainingset/셀트리온_traindata_200.csv')
df_celt=df_celt.head(200)
#df_celt

In [73]:
df_KAL = pd.read_csv('./trainingset/대한항공_testdata_300.csv', header=None)
df_celt=df_celt.head(200)
# df_KAL=df_KAL.head(300)
df_KAL=df_KAL.head(299)
df_KAL.columns = ['id','Title','label']
# df_KAL

In [74]:
df = pd.concat([df_train_skt, df_mobis, df_hb, df_S_Oil,df_IBK,df_celt])

In [75]:
df = df[['id','Title','label']]
df

,id,Title,label
0,0.0,국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?,0
1,1.0,'국정원 해킹 의혹' 상임위 앞두고 與野 전초전,0
2,2.0,인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리,0
3,3.0,SKT LG KT 인터넷가입 비교사이트 ‘펭귄통신’ 설치 당일 현금지원 이벤트,0
4,4.0,설 연휴 고향 가는 길 언제 출발해야 덜 막힐까,0
...,...,...,...
195,1172.0,"원 이퀴티 파트너스 포, 엘.피., 셀트리온헬스케어 주식 631만주 장내매도",-1
196,1630.0,"코스닥, 장 초반 하락세…640선 또 '위태'",-1
197,696.0,“ 3000% ”터질 바이오 단돈 3천원대! 2019년 마지막 新대장 바이오 온다!,1
198,1563.0,"코스닥, 개인 '사자'에 700 향해 ↑",1


In [76]:
from sklearn.model_selection import train_test_split

## 데이터셋 구성
- Train Dataset : skt 200개, 모비스 200개, 현백 200개, 에쓰오일 200개, 기업은행 200개, 셀트리온 200개
- Test Dataset : skt 282개

In [77]:
# Train 데이터와 Test 데이터 분리
X_train = df['Title']
y_train = df['label']
X_test = df_test_skt['Title']
y_test = df_test_skt['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [78]:
# 문장을 나누기 위한 Stopwords 선정
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [79]:
#!pip3 install konlpy

In [80]:
# Sentence Tokenizing

import konlpy
from konlpy.tag import Okt

okt = Okt()
X_train_token = []
for sentence in X_train:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train_token.append(temp_X)

X_test_token = []
for sentence in X_test:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test_token.append(temp_X)

### 단어길이 추출

In [81]:
li_train = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_train_token)):
   li_train.append(len(X_train_token[i]))
max(li_train)

li_test = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_test_token)):
   li_test.append(len(X_test_token[i]))
max(li_test)


66

### max_words를 35000에서 100으로 줄임(test 길이 중 가장 큰것은 66)

In [82]:
from keras.preprocessing.text import Tokenizer
max_words = 100
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_token) 
X_train_token = tokenizer.texts_to_sequences(X_train_token) 
X_test_token = tokenizer.texts_to_sequences(X_test_token)

### One-Hot encoding 2가지 방법 - 2번째 방법 사용 (to_categorize)

In [83]:
# 1. 
import numpy as np
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 0, 1]) 
    elif y_train.iloc[i] == 0:
        y_train_encode.append([0, 1, 0]) 
    elif y_train.iloc[i] == -1:
        y_train_encode.append([1, 0, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 0, 1]) 
    elif y_test.iloc[i] == 0:
        y_test_encode.append([0, 1, 0])
    elif y_test.iloc[i] == -1:
        y_test_encode.append([1, 0, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

In [84]:
len(y_train_encode)

1022

In [85]:
# 2. 
# to_categorical 방법은 0 1 -1 순서로 인코딩된다!
from keras.utils import to_categorical

y_train_cata = to_categorical(y_train, num_classes = 3)
y_test_cata = to_categorical(y_test, num_classes = 3)

In [61]:
y_train_cata

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### max_len 20 to 100으로 변경 ; 단어 길이 고려

In [86]:
max_len = 100 # pad_sequesces이용, 전체 데이터의 길이를 20로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

### 모델 저장1
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


MODEL_DIR = './ModelCheckpoint/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
mdpath = './ModelCheckpoint/{epoch:02d}-{val_loss:.4f}.hdf5'
mc = ModelCheckpoint(filepath=mdpath,monitor='val_acc',mode='max',verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_loss',patience=10,mode='min',verbose=1)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

### 모델 저장2
from keras.models import load_model
##### 모델 컴퓨터에 저장
model.save('모델이름')
##### 모델 불러오기
del model # 테스트를 위해 메모리 내의 모델을 삭제
model = load_model('모델명')

### LSTM 모델

### earlystopping과 modelcheckpoint 추가
### embedding 64로 변경

In [87]:
!pip install tensorflow

In [88]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

Epoch 1/10
110/110 [==============================] - 20s 181ms/step - loss: 1.0156 - accuracy: 0.5259 - val_loss: 1.0142 - val_accuracy: 0.5447
Epoch 2/10
110/110 [==============================] - 20s 179ms/step - loss: 0.9093 - accuracy: 0.5696 - val_loss: 0.8125 - val_accuracy: 0.6748
Epoch 3/10
110/110 [==============================] - 20s 179ms/step - loss: 0.8566 - accuracy: 0.6005 - val_loss: 0.7414 - val_accuracy: 0.7073
Epoch 4/10
110/110 [==============================] - 20s 177ms/step - loss: 0.8247 - accuracy: 0.6197 - val_loss: 0.7374 - val_accuracy: 0.7073
Epoch 5/10
110/110 [==============================] - 20s 181ms/step - loss: 0.8075 - accuracy: 0.6160 - val_loss: 0.6991 - val_accuracy: 0.7236
Epoch 6/10
110/110 [==============================] - 19s 177ms/step - loss: 0.7972 - accuracy: 0.6297 - val_loss: 0.7939 - val_accuracy: 0.6667
Epoch 7/10
110/110 [==============================] - 19s 175ms/step - loss: 0.7843 - accuracy: 0.6342 - val_loss: 0.7652 - val_ac

In [89]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 1s 58ms/step - loss: 0.9475 - accuracy: 0.5979

 테스트 정확도 : 59.79%


## KFOLD 검증

In [90]:
#seed 값 설정
seed = 1
np.random.seed(seed)
#tf.set_random_seed(seed)

In [91]:
X=X_train
Y=y_train_cata

In [92]:
#10개의 파일로 쪼갬
n_fold = 15

In [93]:
skf =KFold(n_splits=n_fold, shuffle=True,random_state=seed)

In [94]:
#빈 accuracy 배열
accuracy =[]

In [95]:
#모델의 설정, 컴파일, 실행
for train, test in skf.split(X,Y):
    from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
    es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
    mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

    model1 = Sequential()
    model1.add(Embedding(max_words, 64))
    model1.add(LSTM(128))
    model1.add(Dense(3, activation='softmax'))
    model1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model1.fit(X[train],Y[train], epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)
    k_accuracy = '%.4f'%(model1.evaluate(X[test],Y[test])[1])
    accuracy.append(k_accuracy)

Epoch 1/10
103/103 [==============================] - 17s 164ms/step - loss: 1.0119 - accuracy: 0.5263 - val_loss: 1.0121 - val_accuracy: 0.5439
Epoch 2/10
103/103 [==============================] - 16s 152ms/step - loss: 0.9175 - accuracy: 0.5556 - val_loss: 0.8029 - val_accuracy: 0.6842
Epoch 3/10
103/103 [==============================] - 15s 148ms/step - loss: 0.8575 - accuracy: 0.6033 - val_loss: 0.7073 - val_accuracy: 0.7456
Epoch 4/10
103/103 [==============================] - 14s 137ms/step - loss: 0.8260 - accuracy: 0.6238 - val_loss: 0.8150 - val_accuracy: 0.6930
Epoch 5/10
103/103 [==============================] - 15s 150ms/step - loss: 0.8089 - accuracy: 0.6345 - val_loss: 0.7488 - val_accuracy: 0.7105
Epoch 6/10
103/103 [==============================] - 16s 152ms/step - loss: 0.7912 - accuracy: 0.6316 - val_loss: 0.7132 - val_accuracy: 0.7018
Epoch 7/10
103/103 [==============================] - 16s 152ms/step - loss: 0.7832 - accuracy: 0.6335 - val_loss: 0.7844 - val_ac

Epoch 2/10
103/103 [==============================] - 15s 141ms/step - loss: 0.9193 - accuracy: 0.5614 - val_loss: 0.7945 - val_accuracy: 0.6842
Epoch 3/10
103/103 [==============================] - 14s 140ms/step - loss: 0.8537 - accuracy: 0.6082 - val_loss: 0.7736 - val_accuracy: 0.6842
Epoch 4/10
103/103 [==============================] - 15s 143ms/step - loss: 0.8307 - accuracy: 0.6072 - val_loss: 0.7433 - val_accuracy: 0.6930
Epoch 5/10
103/103 [==============================] - 14s 137ms/step - loss: 0.8096 - accuracy: 0.6228 - val_loss: 0.7330 - val_accuracy: 0.7193
Epoch 6/10
103/103 [==============================] - 15s 143ms/step - loss: 0.8170 - accuracy: 0.6189 - val_loss: 0.7363 - val_accuracy: 0.7105
Epoch 7/10
103/103 [==============================] - 14s 140ms/step - loss: 0.7812 - accuracy: 0.6326 - val_loss: 0.7787 - val_accuracy: 0.6842
Epoch 8/10
103/103 [==============================] - 14s 141ms/step - loss: 0.7774 - accuracy: 0.6423 - val_loss: 0.7613 - val_ac

Epoch 3/10
103/103 [==============================] - 14s 138ms/step - loss: 0.8544 - accuracy: 0.5945 - val_loss: 0.7919 - val_accuracy: 0.6754
Epoch 4/10
103/103 [==============================] - 14s 136ms/step - loss: 0.8268 - accuracy: 0.6062 - val_loss: 0.7385 - val_accuracy: 0.6930
Epoch 5/10
103/103 [==============================] - 14s 138ms/step - loss: 0.8104 - accuracy: 0.6150 - val_loss: 0.7434 - val_accuracy: 0.6667
Epoch 6/10
103/103 [==============================] - 14s 132ms/step - loss: 0.7928 - accuracy: 0.6306 - val_loss: 0.8075 - val_accuracy: 0.6404
Epoch 7/10
103/103 [==============================] - 14s 137ms/step - loss: 0.7759 - accuracy: 0.6374 - val_loss: 0.8637 - val_accuracy: 0.6404
Epoch 8/10
103/103 [==============================] - 14s 137ms/step - loss: 0.7711 - accuracy: 0.6413 - val_loss: 0.7895 - val_accuracy: 0.6842
Epoch 00008: early stopping
3/3 [==============================] - 0s 13ms/step - loss: 0.9166 - accuracy: 0.5000
Epoch 1/10
103/1

Epoch 4/10
103/103 [==============================] - 14s 139ms/step - loss: 0.8210 - accuracy: 0.6033 - val_loss: 0.8792 - val_accuracy: 0.6609
Epoch 5/10
103/103 [==============================] - 14s 140ms/step - loss: 0.8024 - accuracy: 0.6277 - val_loss: 0.7784 - val_accuracy: 0.6783
Epoch 6/10
103/103 [==============================] - 15s 141ms/step - loss: 0.7804 - accuracy: 0.6423 - val_loss: 0.7769 - val_accuracy: 0.6609
Epoch 7/10
103/103 [==============================] - 15s 144ms/step - loss: 0.7797 - accuracy: 0.6423 - val_loss: 0.7870 - val_accuracy: 0.7043
Epoch 00007: early stopping
3/3 [==============================] - 0s 36ms/step - loss: 0.9702 - accuracy: 0.5309
Epoch 1/10
103/103 [==============================] - 15s 148ms/step - loss: 1.0031 - accuracy: 0.5224 - val_loss: 0.9648 - val_accuracy: 0.6087
Epoch 2/10
103/103 [==============================] - 16s 155ms/step - loss: 0.9062 - accuracy: 0.5595 - val_loss: 0.8139 - val_accuracy: 0.6435
Epoch 3/10
103/1

Epoch 5/10
103/103 [==============================] - 17s 160ms/step - loss: 0.7989 - accuracy: 0.6335 - val_loss: 0.7471 - val_accuracy: 0.7130
Epoch 6/10
103/103 [==============================] - 16s 156ms/step - loss: 0.7902 - accuracy: 0.6170 - val_loss: 0.7531 - val_accuracy: 0.6609
Epoch 7/10
103/103 [==============================] - 16s 160ms/step - loss: 0.7804 - accuracy: 0.6248 - val_loss: 0.7969 - val_accuracy: 0.6522
Epoch 8/10
103/103 [==============================] - 17s 161ms/step - loss: 0.7769 - accuracy: 0.6345 - val_loss: 0.8036 - val_accuracy: 0.6870
Epoch 9/10
103/103 [==============================] - 16s 158ms/step - loss: 0.7658 - accuracy: 0.6316 - val_loss: 0.7810 - val_accuracy: 0.6957
Epoch 00009: early stopping
3/3 [==============================] - 0s 38ms/step - loss: 0.8327 - accuracy: 0.6543
Epoch 1/10
103/103 [==============================] - 17s 165ms/step - loss: 1.0310 - accuracy: 0.5244 - val_loss: 1.0111 - val_accuracy: 0.5478
Epoch 2/10
103/1

In [96]:
print("\n 테스트 정확도 : {:.2f}%".format(model1.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 1s 59ms/step - loss: 0.9758 - accuracy: 0.5872

 테스트 정확도 : 58.72%
